In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
feature_list = ['x','y','hour','weekday','day','month','year', 'accuracy']

In [3]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    #Feature engineering
    fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09] #feature weights
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def run_prediction(df_train, df_test, n_cell_x, n_cell_y, th, kn):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th, kn))
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [5]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th, kn):
    start_time = time.time()
    
    df_cell_train = df_train[((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y))]# |
    '''
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y - 1))]
    '''
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=kn, weights='distance', 
                               metric='manhattan')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [6]:
def run_validation():
    # Run validation
    df_train = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    df_valiation = df_train[df_train.time > 786239 * 0.875]
    
    df_train = df_train[df_train.time <= 786239 * 0.875]
    
    n_cell_x = 10
    n_cell_y = 20 
    
    print('Preparing train data', flush=True)
    df_train = prepare_data(df_train, n_cell_x, n_cell_y)
    df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
    df_valiation['p1'] = np.nan
    df_valiation['p2'] = np.nan
    df_valiation['p3'] = np.nan
    
    df_valiation1 = df_valiation[df_valiation.accuracy / 0.09 < 32]
    df_valiation2 = df_valiation[(df_valiation.accuracy / 0.09 >= 32) & (df_valiation.accuracy / 0.09 < 59)]
    df_valiation3 = df_valiation[(df_valiation.accuracy / 0.09 >= 59) & (df_valiation.accuracy / 0.09 < 68)]
    df_valiation4 = df_valiation[(df_valiation.accuracy / 0.09 >= 68) & (df_valiation.accuracy / 0.09 < 104)]
    df_valiation5 = df_valiation[df_valiation.accuracy / 0.09 >= 104]
    
    prediction_result1 = run_prediction(df_train, df_valiation1, n_cell_x, n_cell_y, 5, k[0])
    prediction_result2 = run_prediction(df_train, df_valiation2, n_cell_x, n_cell_y, 5, k[1])
    prediction_result3 = run_prediction(df_train, df_valiation3, n_cell_x, n_cell_y, 5, k[2])
    prediction_result4 = run_prediction(df_train, df_valiation4, n_cell_x, n_cell_y, 5, k[3])
    prediction_result5 = run_prediction(df_train, df_valiation5, n_cell_x, n_cell_y, 5, k[4])
    
    prediction_result = pd.DataFrame()
    prediction_result = prediction_result.append(prediction_result1)
    prediction_result = prediction_result.append(prediction_result2)
    prediction_result = prediction_result.append(prediction_result3)
    prediction_result = prediction_result.append(prediction_result4)
    prediction_result = prediction_result.append(prediction_result5)
    
    # Calculate score
    prediction_result.sort_index(inplace=True)
    prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
    prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
    prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33
    
    print(prediction_result.score.describe(), flush=True)

In [7]:
k = [35,35,35,35,35]
run_validation()
#0.539376

k = [37,36,35,34,33]
run_validation()
#0.539309

k = [39,37,35,34,33]
run_validation()
#0.539177

k = [45,40,35,30,25]
run_validation()
#0.538709

k = [37,36,35,35,35]
run_validation()
#0.539254

k = [39,37,35,35,35]
run_validation()
#0.539121

k = [45,40,35,35,35]
run_validation()
#0.538660

Preparing train data
Elapsed time overall: 17.174227714538574 seconds 0
Elapsed time overall: 18.96017575263977 seconds 1
Elapsed time overall: 19.381534814834595 seconds 2
Elapsed time overall: 19.06308937072754 seconds 3
Elapsed time overall: 20.324678421020508 seconds 4
Elapsed time overall: 19.86591386795044 seconds 5
Elapsed time overall: 19.193337440490723 seconds 6
Elapsed time overall: 19.954301834106445 seconds 7
Elapsed time overall: 19.583720207214355 seconds 8
Elapsed time overall: 17.436633586883545 seconds 9
Elapsed time overall: 18.485978603363037 seconds 0
Elapsed time overall: 20.51283288002014 seconds 1
Elapsed time overall: 20.553257703781128 seconds 2
Elapsed time overall: 19.31760549545288 seconds 3
Elapsed time overall: 19.29975700378418 seconds 4
Elapsed time overall: 20.56389856338501 seconds 5
Elapsed time overall: 19.831688404083252 seconds 6
Elapsed time overall: 20.558162212371826 seconds 7
Elapsed time overall: 19.88705849647522 seconds 8
Elapsed time overa

In [ ]:
prediction_result

Validation result: 0.539376

In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [ ]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)